In [8]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf, StorageLevel, RDD
from pyspark.sql.functions import col

In [9]:
conf = SparkConf().setAppName('EnumRDDs')\
    .setMaster('yarn')

spark = SparkSession.builder.config(conf=conf).getOrCreate()

24/08/25 21:15:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/25 21:15:53 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/25 21:15:53 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/08/25 21:15:53 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/08/25 21:15:53 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
24/08/25 21:15:53 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [10]:
df = spark.createDataFrame([('Mary', 'Foo', 12), ('Bob', 'Bar', 14)], ['name', 'name2', 'age'])
df.show()

+----+-----+---+
|name|name2|age|
+----+-----+---+
|Mary|  Foo| 12|
| Bob|  Bar| 14|
+----+-----+---+



In [70]:
from pyspark.sql.functions import lit

for key in [False, True]:
        
    for n in [None, 0,1]:
        # take_result = df.take(n)
        # print('take({0}): type: {1}, value: {2})'.format(n, type(take_result), \
        #     take_result))
    
        head_result = df.where(lit(key)).head(n)
        print('head({0}): type: {1}, value: {2})'.format(n, type(head_result), \
            head_result))
    print('\n')

# n=1
# take_0_result = df.take(n)
# print('take({0}) result:\n type: {1}\n value: {2})'.format(n, type(take_0_result), \
#     take_0_result))

# print(type(df.head()))
# df.where('False').take(1)
# df.head(1)

head(None): type: <class 'NoneType'>, value: None)
head(0): type: <class 'list'>, value: [])
head(1): type: <class 'list'>, value: [])


head(None): type: <class 'pyspark.sql.types.Row'>, value: Row(name='Mary', name2='Foo', age=12))
head(0): type: <class 'list'>, value: [])
head(1): type: <class 'list'>, value: [Row(name='Mary', name2='Foo', age=12)])




In [6]:
df.select([ 
    'name', 'age',             # by string
    df.name, df.age,           # by method
    col('name'), col('age'),   # with function col: from pyspark.sql.functions import col 
    df[0], df[2],              # by index of default collection 
    df.columns[0], df.columns[2], # explicitly by index of collection columns
    df.__getattr__('name'), df.__getattr__('age'), # with __getattr__
    df.__getitem__(0), df.__getitem__(2),          # with __getitem__
    *([c for c in df.columns if c in ('age', 'name')]),  # build list from columns with condition
    df.colRegex("`(age)?+.+`") # by colRegex
 ]).show()

+----+---+----+---+----+---+----+---+----+---+----+---+----+---+----+---+----+-----+
|name|age|name|age|name|age|name|age|name|age|name|age|name|age|name|age|name|name2|
+----+---+----+---+----+---+----+---+----+---+----+---+----+---+----+---+----+-----+
|Mary| 12|Mary| 12|Mary| 12|Mary| 12|Mary| 12|Mary| 12|Mary| 12|Mary| 12|Mary|  Foo|
| Bob| 14| Bob| 14| Bob| 14| Bob| 14| Bob| 14| Bob| 14| Bob| 14| Bob| 14| Bob|  Bar|
+----+---+----+---+----+---+----+---+----+---+----+---+----+---+----+---+----+-----+



In [25]:
df.groupBy([df.age]).count().show()

+---+-----+
|age|count|
+---+-----+
| 14|    1|
| 12|    1|
+---+-----+



In [28]:
df.groupBy([df.age, df.name]).count().show()

+---+-----+-----+
|age| name|count|
+---+-----+-----+
| 14|  Bob|    1|
| 12|Alice|    1|
+---+-----+-----+



In [30]:
df.groupBy([col("age")]).count().show()

+---+-----+
|age|count|
+---+-----+
| 14|    1|
| 12|    1|
+---+-----+



In [31]:
df.groupBy([col("age"), col("name")]).count().show()

+---+-----+-----+
|age| name|count|
+---+-----+-----+
| 12|Alice|    1|
| 14|  Bob|    1|
+---+-----+-----+



In [10]:
df[[df.columns[1],df.columns[0]]].show()

+---+-----+
|age| name|
+---+-----+
| 12|Alice|
| 14|  Bob|
+---+-----+



In [18]:
df[[df.__getattr__("age")]].show()

+---+
|age|
+---+
| 12|
| 14|
+---+



In [19]:
df[[df.__getitem__(1)]].show()

+---+
|age|
+---+
| 12|
| 14|
+---+



In [49]:
df.select("age").show()

+---+-----+
|age| name|
+---+-----+
| 12|Alice|
| 14|  Bob|
+---+-----+



In [11]:
df.select("age", "name").show()

+---+-----+
|age| name|
+---+-----+
| 12|Alice|
| 14|  Bob|
+---+-----+



In [13]:
df.select(df.age).show()

+---+
|age|
+---+
| 12|
| 14|
+---+



In [50]:
df.select(df.age, df.name).show()

+---+-----+
|age| name|
+---+-----+
| 12|Alice|
| 14|  Bob|
+---+-----+



In [14]:
df.createOrReplaceTempView("v_children")
spark.sql("select age, name from v_children").show()

+---+-----+
|age| name|
+---+-----+
| 12|Alice|
| 14|  Bob|
+---+-----+



In [71]:
 
spark.stop()